# SQLite Demo

## Establish connection to database

In [3]:
import sqlite3
conn = sqlite3.connect("trial.sql")
print(conn)
cursor = conn.cursor()
print(cursor)

# TA Example

In [15]:
# Deletes the table Tracks if that exists
cursor.execute("DROP TABLE  IF EXISTS TAs")

# Creates tables TAs and Profs
cursor.executescript('''
CREATE TABLE TAs (
    andrewID TEXT PRIMARY KEY UNIQUE NOT NULL,
    name TEXT,
    age INTEGER,
    major TEXT,
    department TEXT
) ;

CREATE TABLE IF NOT EXISTS Profs (
    andrewID TEXT PRIMARY KEY UNIQUE NOT NULL,
    name TEXT,
    classCode TEXT,
    department TEXT
)
''')
conn.commit()

OperationalError: table Profs already exists

## Insert data into table

In [17]:
cursor.execute("INSERT OR IGNORE INTO TAs (andrewID, name, age, major, department) VALUES ('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')")
cursor.execute("INSERT OR IGNORE INTO TAs (andrewID, name, age, major, department) VALUES ('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT')")
cursor.execute("INSERT OR IGNORE INTO TAs (andrewID, name, age, major, department) VALUES ('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS')")
cursor.execute("INSERT OR IGNORE INTO TAs (andrewID, name, age, major, department) VALUES ('william', 'William', 19, 'Computer Science', 'SCS')")
cursor.execute("INSERT OR IGNORE INTO TAs (andrewID, name, age, major, department) VALUES ('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')")
conn.commit()

## [IMPORTANT] What if we do not know what the value is? What if it is a variable instead?

In [13]:
deniz = {
    "andrewID" : "mbirlikc",
    "name" : "Deniz Birlikci",
    "age" : "20",
    "major" : "Artificial Intelligence",
    "department" : "SCS"
}

andrew = {
    "andrewID" : "ayoun2",
    "name" : "Andrew Youn",
    "age" : "20",
    "major" : "Mechanical Engineering",
    "department" : "CIT"
}

# Example for entering data into the table
cursor.execute('''INSERT OR REPLACE INTO TAs 
        (andrewID, name, age, major, department) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        (deniz["andrewID"], deniz["name"], deniz["age"], deniz["major"], deniz["department"]))

# Why do we do the VALUES ( ?, ?, ?, ?, ? ) method? Becuase
# our values can be changing and we are better off with vorking with variables

# We can also use format strings to achieve the same purpose
searchedAndrewID = "mbirlikc"
cursor.execute("""SELECT * FROM TAs WHERE andrewID = (?)""", (searchedAndrewID,))
conn.commit()

## So ... Format Strings & (?, ?, ?), (val1, val2, val3) Format is your BEST FRIEND!!!

# What happens if you try to write Duplicate Table or Duplicate Entries?

You will get an error. 

To prevent that from happening, you have to be a little more careful and specific about what you want.

Say your program wishes to enter the same andrewID again...

    1 - If this is just to update the value, you can write
        "INSERT OR REPLACE INTO ... "
    2 - If you want to just stick with the first entry, and ingore any following, you can write
        "INSERT OR INGORE INTO ... "

Say you want to create a new table. 

Just saying "CREATE TABLE ..." can lead to problems when that table already exists. 

What you can write instead would be:

    - "CREATE TABLE IF NOT EXISTS ... "

### Let's Revisit why our first create table code did not fail

In [43]:


# BEFORE CREATING TABLE TAs IT DELETES THE TABLE IF IT ALREADY EXISTS
cursor.execute("DROP TABLE IF EXISTS TAs")

# Creates tables TAs and Profs
cursor.executescript('''
CREATE TABLE TAs (
    andrewID TEXT PRIMARY KEY UNIQUE NOT NULL,
    name TEXT,
    age INTEGER,
    major TEXT,
    department TEXT
) ;

CREATE TABLE IF NOT EXISTS Profs (
    andrewID TEXT PRIMARY KEY UNIQUE NOT NULL,
    name TEXT,
    classCode TEXT,
    department TEXT
)
''')

# IN THE PROFS EXAMPLE, WE HAVE THE 'IF NOT EXISTS' FLAG WHICH 
# CREATES THE TABLE ONLY IF IT NOT EXISTS

## Read data from table

Components Included

    1 - The select statement
    2 - fetchall() and fetchone() methods
    3 - Iterating through result
    4 - Where clause

## Select, fetchall, and fetchone

In [23]:
cursor.execute("SELECT * FROM TAs")
# singleList = cursor.fetchone()
dataList = cursor.fetchall()
# print("Result of fetchone", singleList)
print("Result of fetchall", dataList)
for row in dataList:
    print(row)

Result of fetchone ('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')
Result of fetchall [('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT'), ('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS'), ('william', 'William', 19, 'Computer Science', 'SCS'), ('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')]
('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT')
('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS')
('william', 'William', 19, 'Computer Science', 'SCS')
('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')


## WHERE clause
The WHERE clause is used to filter records. You can use it to extract only those records that fulfill the specified condition. Conditions can be defined using arithmetic operators and BETWEEN, LIKE, and IN.

In [26]:
coolPeople = cursor.execute("SELECT * FROM TAs WHERE age > 18")
for row in coolPeople:
    print(row)

('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')
('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT')
('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS')
('william', 'William', 19, 'Computer Science', 'SCS')
('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')


In [27]:
ageLimit = 18

votingPeople = cursor.execute("SELECT * FROM TAs WHERE age >= {}".format(ageLimit))
for row in votingPeople:
    print(row)


('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')
('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT')
('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS')
('william', 'William', 19, 'Computer Science', 'SCS')
('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')


## ORDER BY clause

In [29]:
print("oldest to youngest")
oldestFirst = cursor.execute("SELECT * FROM TAs ORDER BY age DESC") #ASC means ascending
for row in oldestFirst:
    print(row)

print("youngest to oldest")
youngestFirst = cursor.execute("SELECT * FROM TAs WHERE department = 'SCS' ORDER BY age ASC") #DESC means descending
for row in youngestFirst:
    print(row)
    

oldest to youngest
('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')
('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')
('ayoun2', 'Andrew', 19, 'Mechanical Engineering', 'CIT')
('wzha', 'Winston', 19, 'CS & Business Double Major', 'Tepper & SCS')
('william', 'William', 19, 'Computer Science', 'SCS')
youngest to oldest
('william', 'William', 19, 'Computer Science', 'SCS')
('mbirlikc', 'Deniz', 20, 'Artificial Intelligence', 'SCS')
('skocabalk', 'Sedef', 20, 'Computer Science', 'SCS')


## Fun operations: MIN/MAX, COUNT, AVG, SUM

In [33]:
oldest = cursor.execute("SELECT MIN(age) FROM TAs")
oldest = cursor.fetchone()
print(oldest)

(19,)


In [34]:
numParticipants = cursor.execute("SELECT COUNT(department) FROM TAs")
for row in numParticipants:
    print(row)

(5,)


In [35]:
avgAge = cursor.execute("SELECT AVG(age) FROM TAs")
for row in avgAge:
    print(row)

(19.4,)


In [36]:
sumAge = cursor.execute("SELECT SUM(age) FROM TAs")
for row in sumAge:
    print(row)

(97,)
